# Guided Backpropagation

- ReLU
- Deconvnet ReLU
- Guided Backpropagation ReLU

![alt_text](./guided_backprop.jpg)

## 1) Import Required Libraries

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.autograd import Function 
from torch.autograd import Variable

## 2) Data

In [2]:
x = torch.Tensor([[3,1,5],[-2,5,-7],[3,2,-4]])

x = Variable(x,requires_grad=True)
y = Variable(2*torch.ones(3,3))

print("Input X:{} \n Target Y:{}".format(x.data,y.data))
print("Gradient of X: {}\n".format(x.grad))

Input X:
 3  1  5
-2  5 -7
 3  2 -4
[torch.FloatTensor of size 3x3]
 
 Target Y:
 2  2  2
 2  2  2
 2  2  2
[torch.FloatTensor of size 3x3]

Gradient of X: None



## 3-1) ReLU

In [3]:
x = torch.Tensor([[3,1,5],[-2,5,-7],[3,2,-4]])

x = Variable(x,requires_grad=True)
y = Variable(2*torch.ones(3,3))

print("Input X:{} \n Target Y:{}".format(x.data,y.data))

out = F.relu(x) 

print("Output: {}".format(out.data))

loss = torch.abs(out-y)
print("Loss: {}".format(loss.data))
loss = torch.sum(loss)
loss.backward()

print(x.grad)

Input X:
 3  1  5
-2  5 -7
 3  2 -4
[torch.FloatTensor of size 3x3]
 
 Target Y:
 2  2  2
 2  2  2
 2  2  2
[torch.FloatTensor of size 3x3]

Output: 
 3  1  5
 0  5  0
 3  2  0
[torch.FloatTensor of size 3x3]

Loss: 
 1  1  3
 2  3  2
 1  0  2
[torch.FloatTensor of size 3x3]

Variable containing:
 1 -1  1
 0  1  0
 1  0  0
[torch.FloatTensor of size 3x3]



## 3-2) MyReLU

http://pytorch.org/tutorials/beginner/examples_autograd/two_layer_net_custom_function.html

In [4]:
class MyReLU(torch.autograd.Function):
    """
    We can implement our own custom autograd Functions by subclassing
    torch.autograd.Function and implementing the forward and backward passes
    which operate on Tensors.
    """

    @staticmethod
    def forward(ctx, input):
        """
        In the forward pass we receive a Tensor containing the input and return
        a Tensor containing the output. ctx is a context object that can be used
        to stash information for backward computation. You can cache arbitrary
        objects for use in the backward pass using the ctx.save_for_backward method.
        """
        ctx.save_for_backward(input)
        return input.clamp(min=0)

    @staticmethod
    def backward(ctx, grad_output):
        """
        In the backward pass we receive a Tensor containing the gradient of the loss
        with respect to the output, and we need to compute the gradient of the loss
        with respect to the input.
        """
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        print("grad_input: ",grad_input)
        grad_input[input < 0] = 0
        return grad_input
    
my_relu = MyReLU.apply

In [5]:
x = torch.Tensor([[3,1,5],[-2,5,-7],[3,2,-4]])

x = Variable(x,requires_grad=True)
y = Variable(2*torch.ones(3,3))

print("Input X:{}".format(x.data))

out = my_relu(x) 

loss = torch.abs(out-y)
#print("Loss: {}".format(loss.data))
loss = torch.sum(loss)
loss.backward()

print(x.grad)

Input X:
 3  1  5
-2  5 -7
 3  2 -4
[torch.FloatTensor of size 3x3]

grad_input:  Variable containing:
 1 -1  1
-1  1 -1
 1  0 -1
[torch.FloatTensor of size 3x3]

Variable containing:
 1 -1  1
 0  1  0
 1  0  0
[torch.FloatTensor of size 3x3]



## 3-3) Deconvnet ReLU

http://pytorch.org/docs/0.3.1/notes/extending.html#extending-torch-autograd

In [6]:
class DeconvReLU(Function):
    @staticmethod
    def forward(ctx,input):
        #ctx.save_for_backward(input)
        return input.clamp(min=0)
    
    @staticmethod
    def backward(ctx,grad_output):
        #input = ctx.saved_tensors
        grad_input = grad_output.clone()
        print("grad input: ",grad_input)
        grad_input[grad_input<0] = 0
        return grad_input
        
deconv_relu = DeconvReLU.apply

In [7]:
x = torch.Tensor([[3,1,5],[-2,5,-7],[3,2,-4]])

x = Variable(x,requires_grad=True)
y = Variable(2*torch.ones(3,3))

print("Input X:{}".format(x.data))

out = deconv_relu(x)

#print("Output: {}".format(out.data))

loss = torch.abs(out-y)
#print("Loss: {}".format(loss.data))
loss = torch.sum(loss)
loss.backward()

print(x.grad)

Input X:
 3  1  5
-2  5 -7
 3  2 -4
[torch.FloatTensor of size 3x3]

grad input:  Variable containing:
 1 -1  1
-1  1 -1
 1  0 -1
[torch.FloatTensor of size 3x3]

Variable containing:
 1  0  1
 0  1  0
 1  0  0
[torch.FloatTensor of size 3x3]



## 3-4) Guided Backpropagation ReLU

In [8]:
class GuidedBackpropRelu(Function):
    @staticmethod
    def forward(ctx,input):
        ctx.save_for_backward(input)
        return input.clamp(min=0)
    
    @staticmethod
    def backward(ctx,grad_output):
        input = ctx.saved_tensors[0]
        grad_input = grad_output.clone()
        print("grad input: ",grad_input)
        grad_input[grad_input<0] = 0
        grad_input[input<0]=0
        return grad_input
    
guided_backprop_relu = GuidedBackpropRelu.apply

In [9]:
x = torch.Tensor([[3,1,5],[-2,5,-7],[3,2,-4]])

x = Variable(x,requires_grad=True)
y = Variable(2*torch.ones(3,3))

print("Input X:{}".format(x.data))

out = guided_backprop_relu(x)

#print("Output: {}".format(out.data))

loss = torch.abs(out-y)
#print("Loss: {}".format(loss.data))
loss = torch.sum(loss)
loss.backward()

print(x.grad)

Input X:
 3  1  5
-2  5 -7
 3  2 -4
[torch.FloatTensor of size 3x3]

grad input:  Variable containing:
 1 -1  1
-1  1 -1
 1  0 -1
[torch.FloatTensor of size 3x3]

Variable containing:
 1  0  1
 0  1  0
 1  0  0
[torch.FloatTensor of size 3x3]

